In [33]:
import pandas as pd
import os

# Trabalhando com as partições

Depois de rodar o script `etl.py`, os dataframes ficarão salvos na pasta out_parquet. Basicamente, eles são particionados com base no `chunksize` especificado.
A partir disso, você tem algumas opções para ler essas partições. Optei por carregar e fazer operações dentro de um loop para cada chunk.
Como exemplo, vamos fazer o mesmo filtro utilizado no `etl_dados.ipynb` para encontrar apenas pessoas com 60 anos ou mais, e junta-las em um dataframe no final.

In [41]:
partes = []
for arquivo in sorted(os.listdir("out_parquet")):
    df = pd.read_parquet(f"out_parquet/{arquivo}")
    if "C008" in df.columns: # C008 é o código da variável de idade
        df["C008"] = pd.to_numeric(df["C008"], errors="coerce")
        idosos = df[df["C008"] >= 60]
        print(f"Total de idosos: {len(idosos)}")
        partes.append(idosos)

df_idosos = pd.concat(partes, ignore_index=True)
print("Final:", df_idosos.shape)
print(df_idosos["C008"].head(5))

Total de idosos: 5385
Total de idosos: 6509
Total de idosos: 7652
Total de idosos: 8233
Total de idosos: 8940
Total de idosos: 6835
Final: (43554, 1088)
0    69.0
1    78.0
2    81.0
3    81.0
4    72.0
Name: C008, dtype: float64


Agrupar todos na memória como fiz não é exatamente eficiente, mas servirá já que não estamos trabalhando com tantas linhas de dado.
Vamos importar o dicionário também para auxiliar nas variáveis:

In [42]:
from dic_base import DicBase

dic = DicBase()
print(dic.code_maps["C008"])

{'000 a 130': 'Idade (em anos)'}


Vamos agora continuar aplicando filtros. A forma como fiz na parte dos idosos foi com:
```python
idosos = df[df["C008"] >= 60]
```
Que é indexação booleana. Ela é muito poderosa, e pode ser utilizada para aplicar vários filtros em seguida. Por exemplo, vamos filtrar todos os homens que sabem ler e escrever e fazem uso de óculos.

In [43]:
print(dic.code_maps["C006"]) # Sexo
print(dic.code_maps["D001"]) # Sabe ler e escrever
print(dic.code_maps["G033"]) # Usa óculos

{'1': 'Homem', '2': 'Mulher'}
{'1': 'Sim', '2': 'Não', '9 ': 'Ignorado'}
{'1': 'Sim', '2': 'Não', '9 ': 'Ignorado'}


##

In [37]:
df_idosos["C006"] = pd.to_numeric(df_idosos["C006"], errors="coerce")
df_idosos["D001"] = pd.to_numeric(df_idosos["D001"], errors="coerce")
df_idosos["G033"] = pd.to_numeric(df_idosos["G033"], errors="coerce")
idosos_aux = df_idosos[(df_idosos["C006"] == 1) & (df_idosos["D001"] == 1) & (df_idosos["G033"] == 1)]
print(idosos_aux.shape)

(10199, 1088)
